---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [57]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

text = pd.Series(doc)
text.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [58]:
len(text)

500

In [59]:
df = text.copy()

In [60]:
#04/20/2009; 04/20/09; 4/20/09; 4/3/09
re1 = df.str.extractall(r'(?P<month>\d?\d)[/-](?P<day>\d?\d)[/-](?P<year>\d{4})\b')
df.drop([i[0] for i in re1.index.values],inplace=True)
re2 = df.str.extractall(r'(?P<month>\d?\d)[/-](?P<day>\d?\d)[/-](?P<year>\d{2})\b')
df.drop([i[0] for i in re2.index.values],inplace=True)


In [61]:
#Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
re3 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)\.? (?P<day>\d?\d),? (?P<year>\d{4})')
df.drop([i[0] for i in re3.index.values],inplace=True)

In [62]:
#20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
re4 = df.str.extractall(r'(?P<day>\d?\d) (?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,\.]? (?P<year>\d{4})')
df.drop([i[0] for i in re4.index.values],inplace=True)

In [63]:
#Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
re5 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<day>\d?\d(?:th|st|nd|rd)), (?P<year>\d{4})')
df.drop([i[0] for i in re5.index.values],inplace=True)

In [64]:
#Feb 2009; Sep 2009; Oct 2010
re6 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*) (?P<year>\d{4})')
df.drop([i[0] for i in re6.index.values],inplace=True)

In [65]:
#6/2008; 12/2009
re7 = df.str.extractall(r'(?P<month>\d?\d)/(?P<year>\d{4})')
df.drop([i[0] for i in re7.index.values],inplace=True)

In [66]:
#2009; 2010
re8 = df.str.extractall(r'(?P<year>\d{4})')
df.drop([i[0] for i in re8.index.values],inplace=True)

In [70]:

month_dict = {'Jan':'1','Feb':'2','Mar':'3','Apr':'4','May':'5','Jun':'6','Jul':'7','Aug':'8','Sep':'9','Oct':'10',
              'Nov':'11','Dec':'12','January':'1','February':'2','March':'3','April':'4','June':'6','July':'7',
              'August':'8','September':'9','October':'10','November':'11','December':'12','Decemeber':'12','Janaury':'1'}
for re in [re3,re4,re5,re6]:
    re['month'] = re['month'].map(month_dict)

for re in [re6,re7,re8]:
    re['day'] = 1
    
re8['month'] = 1 

re2['year'] = re2['year'].apply(lambda x: '19'+x)


In [72]:
seq = ['year','month','day']
result = re1[seq].append(re2[seq]).append(re3[seq]).append(re4[seq]).append(re5[seq]).append(re6[seq]).append(re7[seq]).append(re8[seq])


In [73]:
result['year'] = result['year'].apply(int)
result['month'] = result['month'].apply(int)
result['day'] = result['day'].apply(int)

In [74]:
result.sort_values(['year','month','day'], inplace=True)

In [75]:
def date_sorter():
    
    # Your code here
    
    return pd.Series([i[0] for i in result.index.values])# Your answer here